In [ ]:
# Initializing

!pip install nltk
!pip install -U textblob 
!python -m textblob.download_corpora
!pip install nltk
!pip install openpyxl

import sys
import pandas as pd
import xlwt
import csv
import numpy as np
from numpy import loadtxt
from textblob import TextBlob
from collections import Counter

import nltk, re, string, collections
from nltk import word_tokenize 
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.util import ngrams

In [ ]:
def readFile(fileName):
  file = open(fileName, "r")
  words = file.read()
  file.close
  return words  

In [ ]:
# Filtering

for x in ["20","21","22","23","24","25","26","27"]:

  # appendFile = open('filtered.txt','a+')

  path = "filtered-corpus/filtered-donem" + x + ".txt"
  all_files = glob.glob(path)
  words = ""
  
  # appendFile = open('filtered-corpus/filtered-donem' + x + '.txt','a+')

  for file in all_files:
    words = words + readFile(file)
  
  tokenized = [idx.lower() for idx in words.split() if re.findall("^[a-zA-ZğüşöçİĞÜŞÖÇ]+$", idx) and len(idx) > 1]

  # for idx in tokenized:
  #   appendFile.write(" "+idx.lower())
  
  # appendFile.close()

In [ ]:
# n-grams

for x in ["20","21","22","23","24","25","26","27"]:

  path = "corpus/donem" + x + "/yıl*/*.txt"
  all_files = glob.glob(path)
  words = ""
  
  for file in all_files:
    words = words + readFile(file)
  
  tokenized = [idx.lower() for idx in words.split() if re.findall("^[a-zA-Z0-9ğüşöçİĞÜŞÖÇ]+$", idx) and len(idx) > 1]

  unigrams = ngrams(tokenized, 1)
  unigramsFreq = collections.Counter(unigrams)
  df = pd.DataFrame(unigramsFreq.most_common(1000), columns = ['Words', 'Score'])
  df = df.sort_values(by='Score', ascending=False)
  p = '.results-corpus/without-stopwords/unigrams/unigram'+ x +'.csv'
  df.to_csv(p, index = True, header=True)

  bigrams = ngrams(tokenized, 2)
  bigramsFreq = collections.Counter(bigrams)
  df = pd.DataFrame(bigramsFreq.most_common(1000), columns = ['Words', 'Score'])
  df = df.sort_values(by='Score', ascending=False)
  p = '.results-corpus/without-stopwords/bigrams/bigram'+ x +'.csv'
  df.to_csv(p, index = True, header=True)

  trigrams = ngrams(tokenized, 3)
  trigramsFreq = collections.Counter(trigrams)
  df = pd.DataFrame(trigramsFreq.most_common(1000), columns = ['Words', 'Score'])
  df = df.sort_values(by='Score', ascending=False)
  p = '.results-corpus/without-stopwords/trigrams/trigram'+ x +'.csv'
  df.to_csv(p, index = True, header=True)

In [ ]:
# Collocations

corpus = readFile('filtered-corpus/all-corpus-for-collacations.txt')

tokenized = [idx.lower() for idx in corpus.split()]
# tokenized = [item for items in corpus for item in items]

bigrams = nltk.collocations.BigramAssocMeasures()
trigrams = nltk.collocations.TrigramAssocMeasures()

bigramFinder = nltk.collocations.BigramCollocationFinder.from_words(tokenized)
trigramFinder = nltk.collocations.TrigramCollocationFinder.from_words(tokenized)

bigramFinder.apply_freq_filter(30)
trigramFinder.apply_freq_filter(30)

bigram_freq = bigramFinder.ngram_fd.items()
bigramFreqTable = pd.DataFrame(list(bigram_freq), columns=['bigram','freq']).sort_values(by='freq', ascending=False)
# bigramFreqTable[:10]

trigram_freq = trigramFinder.ngram_fd.items()
trigramFreqTable = pd.DataFrame(list(trigram_freq), columns=['trigram','freq']).sort_values(by='freq', ascending=False)
# trigramFreqTable[:10] 

In [ ]:
# PMI

bigramPMITable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.pmi)), columns=['bigram','PMI']).sort_values(by='PMI', ascending=False)
# bigramPMITable[:10]
p = 'collacations-results/all-corpus-bigram-pmi.csv'
bigramPMITable[:100].to_csv(p, index = True, header=True)

In [ ]:
trigramPMITable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.pmi)), columns=['trigram','PMI']).sort_values(by='PMI', ascending=False)
# trigramPMITable[:10]
p = 'collacations-results/all-corpus-trigram-pmi.csv'
trigramPMITable[:100].to_csv(p, index = True, header=True)

In [ ]:
# t-test

bigramTtable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.student_t)), columns=['bigram','t']).sort_values(by='t', ascending=False)
# bigramTtable[:10]
p = 'collacations-results/all-corpus-bigram-t-test.csv'
bigramTtable[:100].to_csv(p, index = True, header=True)

In [ ]:
trigramTtable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.student_t)), columns=['trigram','t']).sort_values(by='t', ascending=False)
# trigramTtable[:10]
p = 'collacations-results/all-corpus-trigram-t-test.csv'
trigramTtable[:100].to_csv(p, index = True, header=True)

In [ ]:
# Chi-Square

bigramChiTable = pd.DataFrame(list(bigramFinder.score_ngrams(bigrams.chi_sq)), columns=['bigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
# bigramChiTable[:10]
p = 'collacations-results/all-corpus-bigram-chi.csv'
bigramChiTable[:100].to_csv(p, index = True, header=True)

In [ ]:
trigramChiTable = pd.DataFrame(list(trigramFinder.score_ngrams(trigrams.chi_sq)), columns=['trigram','chi-sq']).sort_values(by='chi-sq', ascending=False)
# trigramChiTable[:10]
p = 'collacations-results/all-corpus-trigram-chi.csv'
trigramChiTable[:100].to_csv(p, index = True, header=True)

In [ ]:
#comparison

freq_bi = bigramFreqTable[:15].bigram.values
freq_tri = trigramFreqTable[:15].trigram.values

pmi_bi = bigramPMITable[:15].bigram.values
pmi_tri = trigramPMITable[:15].trigram.values

t_bi = bigramTtable[:15].bigram.values
t_tri = trigramTtable[:15].trigram.values

chi_bi = bigramChiTable[:15].bigram.values
chi_tri = trigramChiTable[:15].trigram.values

In [ ]:
bigramsCompare = pd.DataFrame([freq_bi, pmi_bi, t_bi, chi_bi]).T
bigramsCompare.columns = ['Frequency', 'PMI', 't-test', 'Chi-Sq Test']
# bigramsCompare
p = 'collacations-results/all-corpus-bigram-compare.csv'
bigramsCompare[:100].to_csv(p, index = True, header=True)

In [ ]:
trigramsCompare = pd.DataFrame([freq_tri, pmi_tri, t_tri, chi_tri]).T
trigramsCompare.columns = ['Frequency', 'PMI', 't-test', 'Chi-Sq Test']
# trigramsCompare
p = 'collacations-results/all-corpus-trigram-compare.csv'
trigramsCompare[:100].to_csv(p, index = True, header=True)